In [ ]:
text = "<reasoning> I can use the `execute_python` tool to execute the Python code that decrypts the ROT13 message. If the output is the original flag, then the decryption was successful. Otherwise, I need to adjust the code to correctly decrypt the message. </reasoning>\n\n{\n  \"name\": \"execute_python\",\n  \"inputs\": {\n    \"code\": \"def rot13(s): return ''.join(chr((ord(c) - 65 - 13) % 26 + 65) if c.isalpha() else c for c in s)\\nprint(rot13('cvpbPGS{arkg_gvzr_V\\'yy_gel_2_ebhaqf_bs_ebg13_Ncualgvd}'))\",\n    \"reset\": false\n  }\n}"

In [ ]:
import json

def extract_json_objects(text: str):
    """
    Return every top-level JSON object found in *text*.
    Safe against braces inside strings because it lets the real
    JSON parser (`raw_decode`) do the heavy lifting.
    """
    decoder = json.JSONDecoder()
    idx = 0
    found = []

    while True:
        try:
            idx = text.index('{', idx)          # next candidate
        except ValueError:
            break                               # no more '{' → done

        try:
            obj, end = decoder.raw_decode(text[idx:])
            if isinstance(obj, dict):
                found.append(obj)
            idx += end                          # jump past this object
        except json.JSONDecodeError:
            idx += 1                            # not valid here → try one char later
    return found


In [ ]:
print(text)

In [ ]:
calls = extract_json_objects(text)
calls

In [ ]:
a = "Found answer with \\boxed{flag{}} syntax"
a

In [ ]:
import mcp
import importlib
importlib.reload(mcp)
#a = json.dumps(mcp.list_tools_sync(), indent=2)
#print(a)

In [ ]:

tool_names = [x["name"] for x in mcp.list_tools_sync()]
print(calls)
chat_state = {"messages": []}
if len(calls) >= 1:
    for i in range(len(calls)):
        if "name" in calls[i] and calls[i]["name"] in tool_names:
            print(calls[i]["name"])
            print(calls[i]["inputs"])
            if "reset" in calls[i]["inputs"]:
                calls[i]["inputs"]["reset"] = False
            output = mcp.mcp_call_tool(calls[i]["name"], calls[i]["inputs"])
            chat_state["messages"].append(
                {"role":"tool","content":output}
            )
else:
    # no tool call -> finish
    chat_state["finished"]=True

In [ ]:
output

In [1]:
import mcp

#calls[1]["input"]["reset"] = False
output = mcp.mcp_call_tool("execute_python", {"code": "a = 1; print('hey')",  "reset": False})
output["role"] = "tool"
output["content"][0]["text"].startswith("Output:")
output

{'content': [{'type': 'text', 'text': 'Output: hey\n'}],
 'isError': False,
 'role': 'tool'}

In [ ]:
output

In [ ]:
import mcp

output = mcp.mcp_call_tool("execute_python", {"code": "a",  "reset": True})
output["role"] = "tool"
output

In [ ]:
output = mcp.mcp_call_tool("execute_python", {"code": "print('Reseting MCP server')",  "reset": True})
output["role"] = "tool"
output["content"][0]["text"].startswith("Output:")
output

In [ ]:
output

In [ ]:
chat_state

In [ ]:
asd = {'messages': [{'role': 'system', 'content': 'Cutting Knowledge Date: December 2023\nToday Date: 23 May 2025\n\nWhen you receive a tool call response, use the output to format an answer to the original user question.\n\nYou are a helpful assistant with tool calling capabilities.\n'}, {'role': 'user', 'content': '\nYou are an assistant that can call the run_python tool to compute answers.\nThink step by step inside <reasoning> tags.  \nWhen you need code, respond **only** with a JSON tool call.\nPut your final answer inside \x08oxed{}!\n\n[\n  {\n    "name": "execute_python",\n    "description": "Execute Python code and return the output. Variables persist between executions.",\n    "inputSchema": {\n      "type": "object",\n      "properties": {\n        "code": {\n          "type": "string",\n          "description": "Python code to execute"\n        },\n        "reset": {\n          "type": "boolean",\n          "description": "Reset the Python session (clear all variables)",\n          "default": false\n        }\n      },\n      "required": [\n        "code"\n      ]\n    }\n  },\n  {\n    "name": "list_variables",\n    "description": "List all variables in the current session",\n    "inputSchema": {\n      "type": "object",\n      "properties": {}\n    }\n  },\n  {\n    "name": "install_package",\n    "description": "Install a Python package using uv",\n    "inputSchema": {\n      "type": "object",\n      "properties": {\n        "package": {\n          "type": "string",\n          "description": "Package name to install (e.g., \'pandas\')"\n        }\n      },\n      "required": [\n        "package"\n      ]\n    }\n  }\n]\n\nQuestion: A hacker split the access code and scrambled it with their SplitNumberMix cipher. Decrypt  7777777 fla 7777777 g{xthbhjrr} 7777777  to retrieve the original code.\n\nThe flag format is flag{...}\n\nThe flag format is flag{...}\nHint: Numbers are used to break up the flag. The original flag uses only letters.\n'}], 'initial_length': 472}
asd["messages"]

In [ ]:
import pandas as pd
from datasets import Dataset

data_path = "/home/mlajos/CyberML2024/procedural_ctf_generation/challenges/1000_verified/all_challenges.csv"

df = pd.read_csv(data_path)
df = df.rename(columns={"flag": "answer"})
df["question"] = df["question"].str.cat(df["necessary_info"].fillna(""), sep="\n\n")
df["question"] = df["question"].str.cat(df["hint"].fillna(""), sep="\nHint: ")   # add the hint
ds = Dataset.from_pandas(df)
ds = ds.rename_column("question", "prompt")
train_dataset = ds.shuffle(seed=42)
train_dataset

In [ ]:
train_dataset["answer"]
from collections import Counter

types = Counter(type(a) for a in train_dataset["answer"])
print(types)

In [ ]:
import json
import mcp
a = json.dumps(mcp.list_tools_sync(), indent=2)
print(a)

In [ ]:
chat = {
    "utc_ts": "2025-05-28T11:49:34.260882",
    "conversation": {
        "messages": [
            {
                "role": "system",
                "content": "\nYou are a helpful assistant with tool calling capabilities.\n"
            },
            {
                "role": "user",
                "content": "\nYou are an assistant with access to the tools listed below.\n\n### Rules (follow **all** of them) ###\n1. Do your chain-of-thought inside **<reasoning> \u2026 </reasoning>** tags.\n2. **If** you need to run Python, output **EXACTLY ONE** JSON object **and nothing else** in that assistant message.  \n   The object *must* have the shape\n\n        {\n          \"name\": \"execute_python\",\n          \"inputs\": {\n            \"code\": \"print('hello')\",\n            \"reset\": false\n          }\n        }\n\n   \u2013 no prose before or after, no Markdown formatting.\n3. After the tool reply arrives (role `\"tool\"`), read its `Output:` and then answer the\n   question.  Wrap the final flag in **\\boxed{flag{\u2026}}**.  \n   **Never** guess the flag before you have seen it in a tool response.\n4. **Never** include \boxed{flag{\u2026}} in the SAME assistant message that\n   contains a JSON tool call.  Only write the boxed flag **after** the\n   tool\u2019s reply (role=\"tool\") has appeared.\n5. Every assistant message must obey these rules.\n\n### Available tools ###\n[\n  {\n    \"name\": \"execute_python\",\n    \"description\": \"Execute Python code and return the output. Variables persist between executions.\",\n    \"inputSchema\": {\n      \"type\": \"object\",\n      \"properties\": {\n        \"code\": {\n          \"type\": \"string\",\n          \"description\": \"Python code to execute\"\n        },\n        \"reset\": {\n          \"type\": \"boolean\",\n          \"description\": \"Reset the Python session (clear all variables)\",\n          \"default\": false\n        }\n      },\n      \"required\": [\n        \"code\"\n      ]\n    }\n  },\n  {\n    \"name\": \"list_variables\",\n    \"description\": \"List all variables in the current session\",\n    \"inputSchema\": {\n      \"type\": \"object\",\n      \"properties\": {}\n    }\n  },\n  {\n    \"name\": \"install_package\",\n    \"description\": \"Install a Python package using uv\",\n    \"inputSchema\": {\n      \"type\": \"object\",\n      \"properties\": {\n        \"package\": {\n          \"type\": \"string\",\n          \"description\": \"Package name to install (e.g., 'pandas')\"\n        }\n      },\n      \"required\": [\n        \"package\"\n      ]\n    }\n  }\n]\n\nQuestion: An old computer stores the password in hex. Can you decode this hex-encoded secret? Ciphertext: 666c61677b6a6c6673326a64307d\n\nThe flag format is flag{...}\n\nThe flag format is flag{...}\nHint: Hex encoding represents bytes as pairs of 0-9 and a-f.\n"
            },
            {
                "role": "assistant",
                "content": "\n\n<reasoning> Since the hint mentions that hex encoding represents bytes as pairs of 0-9 and a-f, this means the hexadecimal string is encoded in bytes, which we can decode with Python. I can use the execute_python tool to run Python code to decode the hex string. I'm going to use the decode function from the binascii module to decode the string. </reasoning>\n\n{\n  \"name\": \"execute_python\",\n  \"inputs\": {\n    \"code\": \"import binascii; print(binascii.unhexlify('666c61677b6a6c6673326a64307d'))\",\n    \"reset\": false\n  }\n}"
            },
            {
                "content": [
                    {
                        "type": "text",
                        "text": "Output:\nb'flag{jlfs2jd0}'\n"
                    }
                ],
                "isError": False,
                "role": "tool"
            },
            {
                "role": "assistant",
                "content": "\n\n\\boxed{flag{jlfs2jd0}}"
            },
            {
                "role": "system",
                "content": "Found answer with \boxed{flag{}} syntax"
            }
        ],
        "initial_length": 629,
        "finished": True
    }
}

In [ ]:
def _last_assistant_text(chat):
    for msg in reversed(chat["messages"]):
        if msg["role"] == "assistant":
            return msg["content"]


In [ ]:
parsed = _last_assistant_text(chat["conversation"])

In [ ]:
import re

FLAG_BOX_RE  = re.compile(r"\\boxed\{flag\{[^}]+\}\}", flags=re.IGNORECASE)

def parse_flag_from_text(text):
    flag = re.findall(FLAG_BOX_RE, text)
    if not flag:
        return None
    return flag[-1]

In [ ]:
parse_flag_from_text(parsed)

In [ ]:
from helpers import get_model

model, tokenizer = get_model("meta-llama/Llama-3.1-8B-Instruct")#, "/home/mlajos/AutoDidact/full_local_training/Llama_8B_no_hint_251/checkpoint-251")

In [ ]:
import mcp

MCP_TOOL_LIST = mcp.list_tools_sync

In [ ]:
q = "Question: An old terminal reveals: \"Coordinates are hidden with a shift in the code.\" Can you decode the message below? lrgm\u0081oyhn}\u0080mr\u0083\n\nThe flag format is flag{...}"

In [ ]:
import torch, random
from vllm import SamplingParams

def set_seed(seed=0):
    torch.manual_seed(seed); random.seed(seed)

prompt = f"""
You are an assistant with access to the tools listed below.

### Rules (follow **all** of them) ###
1. Do your chain-of-thought inside **<reasoning> … </reasoning>** tags.
2. **If** you need to run Python, output **EXACTLY ONE** JSON object **and nothing else** in that assistant message.  
   The object *must* have the shape

        {{
          "name": "execute_python",
          "inputs": {{
            "code": "print('hello')",
            "reset": false
          }}
        }}

   – no prose before or after, no Markdown formatting.
3. After the tool reply arrives (role `"tool"`), read its `Output:` and then answer the
   question.  Wrap the final flag in **\\boxed{{flag{{…}}}}**.  
   **Never** guess the flag before you have seen it in a tool response.
4. **Never** include \boxed{{flag{{…}}}} in the SAME assistant message that
   contains a JSON tool call.  Only write the boxed flag **after** the
   tool’s reply (role="tool") has appeared.
5. Every assistant message must obey these rules.

### Available tools ###
{MCP_TOOL_LIST}

Question: {q}
"""
set_seed(42)
model.enable_adapter_layers()
txt_base = model.fast_generate([prompt], sampling_params=SamplingParams(
                    temperature=0.6,
                    top_p=0.9,
                    max_tokens=2048
                ))[0].outputs[0].text

set_seed(42)
model.disable_adapter_layers()
txt_base_2 = model.fast_generate([prompt], sampling_params=SamplingParams(
                    temperature=0.6,
                    top_p=0.9,
                    max_tokens=2048
                ))[0].outputs[0].text

print(txt_base)
print(txt_base_2)


In [ ]:
len(txt_base_2)

In [ ]:
lora == txt_base_2

In [ ]:
from peft.utils import get_peft_model_state_dict
print(len(get_peft_model_state_dict(model)))        # should be > 0


In [ ]:
prompt = "Explain why the sky is blue in one sentence."

def gen(use_lora: bool):
    if use_lora:
        model.enable_adapters()
    else:
        model.disable_adapters()
    torch.manual_seed(42)
    return model.fast_generate([prompt])[0].outputs[0].text

print("LoRA :", gen(True))
print("Base :", gen(False))
